In [1]:
import os

if not os.path.exists("lichess_db_puzzle.csv.zst"):
    !wget https://database.lichess.org/lichess_db_puzzle.csv.zst
if not os.path.exists("lichess_db_puzzle.csv"):
    !zstd -d lichess_db_puzzle.csv.zst

In [6]:
import pandas as pd
import chess

# Read CSV and keep only important columns
df = pd.read_csv("lichess_db_puzzle.csv")
df = df[["FEN", "Moves", "Rating", "Themes"]]



In [10]:
def fen_play_first_move_and_force_white(fen, moves):
    board = chess.Board(fen)
    first_move = moves.split()[0]
    # Play the first move of the puzzle, actually setting it up
    board.push_uci(first_move)
    
    # Flip the board if it is black to move
    if board.turn == chess.BLACK:
        board.apply_mirror()
    
    # Return just the FEN of the board
    return board.board_fen()

df["white_to_play_FEN"] = df.apply(lambda r: fen_play_first_move_and_force_white(r["FEN"], r["Moves"]), axis=1)

In [12]:
# Create one hot encoding of the themes
one_hots = df["Themes"].str.get_dummies(sep=" ")
df = df.join(one_hots)

In [14]:
# 80/20 split train/test
train = df.sample(frac=0.8, random_state=1337)
test = df.drop(train.index)

train.to_csv("lichess_db_puzzle_train.csv")
test.to_csv("lichess_db_puzzle_test.csv")